In [ ]:
# Cell 1: Setup
%load_ext autoreload
%autoreload 2

import sys
import gc
import numpy as np
import pandas as pd
from pathlib import Path

# Setup path
sys.path.append(str(Path.cwd().parent))
from src import config, preprocessing, autoencoder, rf_classifier, evaluation, utils

exp_paths = utils.setup_experiment_folder()
print(f"📂 Saving results to: {exp_paths['root']}")

# ==============================================================================
# CHIẾN LƯỢC B: ADAPTIVE TRAINING (Train AE on Combined Source + Target)
# Kịch bản chính: Train 2017 (Labeled) + 2018 (Unlabeled) -> Test 2018
# ==============================================================================

# Cell 2: Load Data (Cả 2 năm)
print("\n--- 1. LOADING DATA ---")

# Load 2017 (Source - Có nhãn để train RF)
df_source, y_source = preprocessing.load_single_dataset_year('2017', binary_mode=True)
X_source = df_source.values
print(f"Source (2017): {X_source.shape}")

# Load 2018 (Target - Giả vờ không có nhãn để train AE, dùng nhãn để test sau cùng)
df_target, y_target = preprocessing.load_single_dataset_year('2018', binary_mode=True)
X_target = df_target.values
print(f"Target (2018): {X_target.shape}")

# Cell 3: Scaling (Fit trên Source, Transform cả hai)
print("\n--- 2. SCALING ---")
scaler = preprocessing.get_scaler()
# Vẫn tuân thủ nguyên tắc: Chỉ fit trên Source (để mô phỏng thực tế)
# Hoặc: Fit trên Combined nếu coi đây là bài toán Transductive Learning (Thường cho kết quả tốt hơn)
# Ở đây ta chọn cách an toàn: Fit Source
X_source_scaled = scaler.fit_transform(X_source)
X_target_scaled = scaler.transform(X_target)
print("✅ Scaling done.")

# Cell 4: Combine Data for Autoencoder (Unsupervised)
print("\n--- 3. PREPARING DATA FOR ADAPTIVE AE ---")
# Gộp Source và Target lại để AE học cấu trúc chung
# Lưu ý: Không dùng nhãn y ở đây
X_combined_scaled = np.concatenate([X_source_scaled, X_target_scaled], axis=0)
print(f"Combined Data Shape: {X_combined_scaled.shape}")

# Xóa biến df để tiết kiệm RAM (vì X_combined rất lớn)
del df_source, df_target, X_source, X_target
gc.collect()

# Cell 5: Train Adaptive Autoencoder
print("\n--- 4. TRAIN ADAPTIVE AUTOENCODER ---")
ae_model = autoencoder.DeepAutoencoder(
    input_dim=config.AE_INPUT_DIM, 
    latent_dim=config.AE_LATENT_DIM,
    hidden_layers=config.AE_HIDDEN_LAYERS
)

# Train trên tập gộp (Thời gian train sẽ lâu gấp đôi)
ae_save_path = exp_paths['models'] / "ae_adaptive_combined.pth"
# Tăng batch size nếu RAM cho phép để train nhanh hơn
ae_model = autoencoder.train_ae(ae_model, X_combined_scaled, save_path=ae_save_path)

# Cell 6: Extract Features (Dùng AE vừa học chung)
print("\n--- 5. EXTRACTING ADAPTIVE LATENT FEATURES ---")
# Lúc này AE đã hiểu cả 2017 và 2018
Z_source = autoencoder.extract_features(ae_model, X_source_scaled)
Z_target = autoencoder.extract_features(ae_model, X_target_scaled)

print(f"Z Source: {Z_source.shape}")
print(f"Z Target: {Z_target.shape}")

# Xóa X_combined để giải phóng RAM cực lớn
del X_combined_scaled
gc.collect()

# Cell 7: Feature Fusion
print("\n--- 6. FEATURE FUSION ---")
# Lấy mRMR features
all_features = config.SELECTED_FEATURES
mrmr_indices = [all_features.index(f) for f in config.mRMR_FEATURES]

# Cắt mRMR từ Scaled Data
X_source_mrmr = X_source_scaled[:, mrmr_indices]
X_target_mrmr = X_target_scaled[:, mrmr_indices]

# Fusion
X_source_fusion = np.hstack([X_source_mrmr, Z_source])
X_target_fusion = np.hstack([X_target_mrmr, Z_target])

print(f"Fusion Source (Train RF): {X_source_fusion.shape}")
print(f"Fusion Target (Test RF):  {X_target_fusion.shape}")

# Cell 8: Train RF on Source (2017) with Adaptive Features
print("\n--- 7. TRAIN CLASSIFIER (RF) ---")
rf_save_path = exp_paths['models'] / "rf_adaptive.joblib"
# Train trên Source (có nhãn)
rf_adaptive = rf_classifier.train_rf(X_source_fusion, y_source, save_path=rf_save_path)

# Cell 9: Evaluate on Target (2018)
print("\n--- 8. EVALUATE ON TARGET (2018) ---")
adaptive_metrics = evaluation.evaluate_model(
    rf_adaptive, 
    X_target_fusion, 
    y_target, 
    save_dir=exp_paths['figures'],
    dataset_name="Adaptive Hybrid (Train 17+18_unlab -> Test 18)"
)

# ==============================================================================
# SO SÁNH VỚI BASELINE (PURE RF - TRAIN 17 -> TEST 18)
# Lưu ý: Baseline vẫn giữ nguyên (chỉ train trên 2017 gốc), không đổi.
# Ta lấy lại kết quả Baseline từ thực nghiệm trước hoặc chạy lại nhanh.
# ==============================================================================

print("\n🥊 SO SÁNH: ADAPTIVE HYBRID vs. BASELINE (Pure RF) 🥊")

# Train lại Baseline (cho công bằng, đảm bảo cùng môi trường hiện tại)
print("Retraining Baseline (Pure RF on Source 2017 mRMR)...")
rf_baseline = rf_classifier.train_rf(
    X_source_mrmr, 
    y_source, 
    save_path=exp_paths['models'] / "rf_baseline_repro.joblib"
)

baseline_metrics = evaluation.evaluate_model(
    rf_baseline, 
    X_target_mrmr, 
    y_target, 
    save_dir=None,
    dataset_name="Baseline (Pure RF)"
)

# Bảng so sánh
ada_acc = adaptive_metrics['accuracy']
base_acc = baseline_metrics['accuracy']
ada_rec = adaptive_metrics['report']['Attack']['recall']
base_rec = baseline_metrics['report']['Attack']['recall']

print("\n📊 === KẾT QUẢ CUỐI CÙNG (STRATEGY B) ===")
print(f"{'Metric':<20} | {'Baseline':<15} | {'Adaptive Hybrid':<15} | {'Improvement':<15}")
print("-" * 80)
print(f"{'Accuracy':<20} | {base_acc:.4f}{'':<9} | {ada_acc:.4f}{'':<9} | {ada_acc - base_acc:+.4f}")
print(f"{'Recall (Attack)':<20} | {base_rec:.4f}{'':<9} | {ada_rec:.4f}{'':<9} | {ada_rec - base_rec:+.4f}")

# Vẽ biểu đồ
import matplotlib.pyplot as plt
names = ['Accuracy', 'Recall (Attack)']
base_vals = [base_acc, base_rec]
ada_vals = [ada_acc, ada_rec]

x = np.arange(len(names))
width = 0.35
fig, ax = plt.subplots()
ax.bar(x - width/2, base_vals, width, label='Baseline', color='gray')
ax.bar(x + width/2, ada_vals, width, label='Adaptive Hybrid', color='green')
ax.set_xticks(x); ax.set_xticklabels(names); ax.legend()
ax.set_title("Effect of Domain Adaptation (Strategy B)")
plt.savefig(exp_paths['figures'] / "strategy_B_comparison.png")
plt.show()